# Example of reading tracker Trees into a pandas data frame for Python analysis 🖥
## See here more details: https://cdcvs.fnal.gov/redmine/projects/gm2analyses/wiki/Python-based_EDM_analysis

### First import the necessary modules

In [1]:
import sys,os
sys.path.append(os.environ["JUPYROOT"]) # where JUPYROOT is path to ROOT e.g. /usr/local/Cellar/root/6.18.04/lib/root
# for Python3 install root_numpy with "python3 -m pip install --user root_numpy"
import root_numpy # see http://scikit-hep.org/root_numpy/install.html  
# for Python3 install root_pandas with "python3 -m pip install --user root_pandas"
from root_pandas import read_root # see https://github.com/scikit-hep/root_pandas 

Welcome to JupyROOT 6.18/04


## Read in the Trees into a pandas data frame

### As we have two new Tress now, QualityTracks and QualityVertices, we need to specify which one to open 

this takes ~2min to load for 35M tracks. Other operatios takes seconds, once data is in the memeory

In [2]:
data = read_root('DATA/Trees/60h_all_quality_tracks.root', 'QualityTracks')

### Get a quick glimpse of data (head and tail) we got over a billion numbers in this table

In [3]:
data

,runNum,subRunNum,eventNum,islandNum,trackMomentum,trackMomentumX,trackMomentumY,trackMomentumZ,trackMomentumUnc,decayVertexPosX,...,nUHits,nVHits,missedLayersFrac,minDriftTime,maxDriftTime,maxResidual,extrapolatedDistance,passCandidateQuality,passTrackQuality,passVertexQuality
0,15921,16,3,19,1712.727539,512.199341,8.262894,-1634.325317,57.717182,-7101.560059,...,6,6,0.000000,8.431014,51.623920,0.175800,0.016023,True,True,False
1,15921,16,3,32,1488.187256,353.067383,-30.994150,-1445.366455,10.332804,-7042.594238,...,10,9,0.095238,10.963304,59.077667,0.227000,0.014446,True,True,True
2,15921,16,3,33,538.117676,245.239731,0.646138,-478.986115,6.021589,-7136.965820,...,6,6,0.000000,13.241358,56.658409,0.092637,0.007120,True,True,True
3,15921,16,3,53,1140.483154,352.003021,-13.173466,-1084.722168,12.152552,-7117.277832,...,6,8,0.125000,5.510454,49.254845,0.228639,0.011527,True,True,True
4,15921,16,3,55,1769.361206,412.681458,-22.436968,-1720.415527,12.600249,-6987.538086,...,12,8,0.090909,13.612383,55.279339,0.269051,0.020338,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35143514,15991,149,154,193,2784.667969,2741.797363,-7.199627,486.693665,36.911140,-3957.141846,...,8,9,0.150000,15.595004,59.157608,0.248354,0.054245,True,True,False
35143515,15991,149,154,217,2124.116455,2071.901855,23.592415,467.480194,28.891079,-1140.763672,...,8,6,0.263158,9.857056,55.896168,0.214529,0.020785,True,True,False
35143516,15991,149,154,222,2924.438965,2900.271973,0.912041,375.186523,58.776600,-3041.445557,...,8,6,0.263158,6.991961,54.695763,0.233597,0.042901,True,True,False
35143517,15991,149,154,227,2744.483643,2710.265869,4.290727,432.007324,43.990440,-2496.093262,...,8,7,0.250000,8.392237,58.666668,0.185277,0.036295,True,True,False


### Also useful to check types of columns (i.e. Ntuples) in our data

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35143519 entries, 0 to 35143518
Data columns (total 35 columns):
runNum                  int32
subRunNum               int32
eventNum                int32
islandNum               int32
trackMomentum           float32
trackMomentumX          float32
trackMomentumY          float32
trackMomentumZ          float32
trackMomentumUnc        float32
decayVertexPosX         float32
decayVertexPosY         float32
decayVertexPosZ         float32
decayVertexMomX         float32
decayVertexMomY         float32
decayVertexMomZ         float32
decayVertexUncR         float32
decayVertexUncY         float32
decayVertexUncPR        float32
decayVertexUncPY        float32
trackT0                 float32
decayTime               float32
hitVolume               bool
trackPValue             float32
station                 int32
nHits                   int32
nUHits                  int32
nVHits                  int32
missedLayersFrac        float32
minDrift

In [ ]:
# Select by column name (NTuple name)
data.loc[:, ["runNum", "subRunNum", "eventNum", "station", "trackT0", "trackMomentum", "nHits", "trackPValue", ]]

# Now let's plot something!

## Select momentum (selecting by column name, which is the NTuple name) for all tracks, and plot for 1M

In [5]:
p = data['trackMomentum']
p_mean = p.mean()
ax = p[0:int(1e6)].plot.hist(bins=250, color="green")
ax.set_xlabel(r"$p$ [MeV]")
print("We have", "{:.2e}".format(len(p)), "tracks in this array, with the mean momentum of ", p_mean, "MeV")
print("Plotting for 1M:")
fig = ax.get_figure()
ax.set_xlim(0, 3500)
# fig.savefig('p.png', dpi=300)

We have 3.51e+07 tracks in this array, with the mean momentum of  1599.076 MeV
Plotting for 1M:


(0, 3500)

## See here for more info on using pandas:
http://pandas.pydata.org/Pandas_Cheat_Sheet.pdf 

https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

### Now get the track time (in $\mu s$) from data 

In [ ]:
t = data['trackT0']*1e-3  # ns -> us 

## Make a 2D histogram wth 100 X by 100 Y bins with 1M tracks

In [ ]:
import matplotlib.pyplot as plt # import plotting module
import numpy as np # import fast array and maths module

#get 1M entries 
t_plot=t[0:int(1e6)]
p_plot=p[0:int(1e6)]

def p_vs_time(t_plot, p_plot, XYbins=(400, 800)):
    fig, ax = plt.subplots()
    h, x, y, image = plt.hist2d(t_plot, p_plot, bins=XYbins, cmap=plt.cm.jet, cmin=10)
    fig.set_size_inches(8, 8)

    font_size=16 # set a global constans
    # Pretify plot (can be part of a function to avoid copying code)
    ax.set_xlabel(xlabel=r"Track time [$\mathrm{\mu}$s]", fontsize=font_size)
    ax.set_ylabel(ylabel=r"$p$ [MeV]", fontsize=font_size)
    ax.tick_params(axis='x', which='both', bottom=True, top=True, direction='inout')
    ax.tick_params(axis='y', which='both', left=True, right=True, direction='inout')
    ax.minorticks_on()
    plt.xticks(fontsize=font_size-1)
    plt.yticks(fontsize=font_size-1)
    ax.set_ylim(0, 3700)
    ax.set_xlim(0, 100)
    cb = plt.colorbar(use_gridspec=False, orientation="vertical", shrink=0.95)
    return fig, ax
    
fig, ax = p_vs_time(t_plot, p_plot)

## Wait?! What about some statisics?

### Let's define some helper functions

In [ ]:
# Put some statisitcs for momentum
# define functions to use later 
from scipy import stats 
def stats5(data):
    '''
    Input is a 1D array 
    '''
    N = len(data)
    mean = np.mean(data)
    meanE = stats.sem(data)
    sd = np.std(data)
    sdE = np.sqrt(sd**2/ (2*N) ) # ROOT-style Gaussian error on the sigma https://root.cern.ch/doc/master/TH1_8cxx_source.html#l07111
    return N, mean, meanE, sd, sdE

# define a legedn function with good formatting and precision of 4 decimal places
def legend5(N, mean, meanE, sd, sdE, prec=4):
    '''
    form a string from 5 stats inputs with given precision
    '''
    meanS=r"$\mathrm{\mu}$"
    sigmaS=r"$\sigma$"
    # form raw string with Latex
    legend = "N={0:.2e}".format(N)+"\n"+str(meanS)+"={0:.{prec}f}({1:d})\n".format(mean, int(round(meanE*10**prec)), prec=prec)+str(sigmaS)+"={0:.{prec}f}({1:d})".format(sd, int(round(sdE*10**prec)), prec=prec)
    return legend

def textL(ax, x, y, legend, font_size=14):
    '''
    return a good formatted plot legend
    '''
    return ax.text(x, y, str(legend),  fontsize=font_size, transform=ax.transAxes, horizontalalignment='center', verticalalignment='center')

## Get nicely-formatted stats and put on the plot

In [ ]:
fig, ax = p_vs_time(t_plot, p_plot) # re-draw our prevous plot 
N, mean, meanE, sd, sdE = stats5(p_plot)
legend_p = legend5(N, mean, meanE, sd, sdE, prec=1)
textL(ax, 0.75, 0.89, "$p$ [MeV]:"+"\n"+str(legend_p), font_size=16)
plt.savefig("py_vs_t_30us.png", dpi=300)

### Now, let's create a cut on track time 

In [ ]:
time_cut = (t > 30)  # us, define a time_cut with time > 30 us

## Create a new data frame where all entries have time above $30 \mu s$

In [ ]:
data_above30us=data[time_cut] 

In [ ]:
print("Our time cut removed", "{:.2e}".format(data.shape[0] - data_above30us.shape[0]), "tracks")

## Redo our previous plot with the time cut

In [ ]:
#get 1M entries from the new data frame where time cut was applied
t_plot=data_above30us['trackT0'][0:int(1e6)] * 1e-3 # ns -> us 
p_plot=data_above30us['trackMomentum'][0:int(1e6)] 
fig, ax = p_vs_time(t_plot, p_plot) # re-draw our prevous plot 
N, mean, meanE, sd, sdE = stats5(p_plot)
legend_p = legend5(N, mean, meanE, sd, sdE, prec=1)
textL(ax, 0.75, 0.89, "$p$ [MeV]:"+"\n"+str(legend_p), font_size=16)

### Create cuts based on stations

In [ ]:
s12_cut = (data['station']==12)
s18_cut = (data['station']==18)
data_above30us_s12=data_above30us[s12_cut]
data_above30us_s18=data_above30us[s18_cut]
print("S12 tracks:", data_above30us_s12.shape[0], "S18 tracks:", data_above30us_s18.shape[0])

# Instructions on fitting coming soon!

### Happy tracking! 🤠